## 关系对齐

In [ ]:
using StringDistances
using Distances

In [ ]:
using DataStructures
include("../src/xlsx.jl")
include("../src/regexpr.jl")
include("../src/extractdata.jl")
include("../src/translatedata.jl")
mkpath("ILLs")

### 相关函数

In [2]:
function get_triple_sign(line)
    triple = split(strip(line), ' ')
    subsign, relsign, objsign = "en", "en", "en"
    sub = get_txt(en_obj_reg, triple[1])
    rel = get_txt(en_rel_reg, triple[2])
    obj = get_txt(en_obj_reg, triple[3])
    isempty(sub) && (sub = get_txt(zh_obj_reg, triple[1]);subsign = "zh")
    isempty(rel) && (rel = get_txt(zh_rel_reg, triple[2]);relsign = "zh")
    isempty(obj) && (obj = get_txt(zh_obj_reg, triple[3]);objsign = "zh")
    [(sub, rel, obj), (subsign, relsign, objsign)]
end
function count_list(list)
    c_list = counter(list)
    sort(Dict(c_list), by=i->c_list[i], rev=true)
end

count_list (generic function with 1 method)

In [4]:
length(train_ques_ner)

14077

In [11]:
0.0007 * 1500 

1.05

In [ ]:
c = count_list(first.(train_ques_ner))

### 测试

In [26]:
que = "who is the leader of the administrative region to which NER belongs?"
trained = unique(train_ques)
que ∈ trained

false

In [30]:
sort(trained, by=x->evaluate(Levenshtein(), que, x)) # 前四个首跳均为 body

9849-element Vector{SubString{String}}:
 "what is the title of the administrative region to NER belongs?"
 "who makes the law of the administrative region to NER belongs?"
 "what titles does the administrative region to which NER belongs?"
 "what is the legislature of the administrative region to NER belongs?"
 "what is the seat title of the administrative region to NER belongs?"
 "which title does the administrative region to NER belong to?"
 "what is the legislature of the " ⋯ 20 bytes ⋯ "n for which NER is responsible?"
 "which event establishes the administrative region to NER belongs?"
 "what is the zip code of the district (division) to which NER belongs"
 "what is the division of the division (division) to which NER belongs"
 "what is the division of the adm" ⋯ 21 bytes ⋯ " the area to which NER belongs?"
 "who is the leader of the legislature where the NER is held?"
 "what is the east of the administrative area responsible for NER?"
 ⋮
 "do you know what is the organiz" ⋯ 143 b

In [ ]:
evaluate(Levenshtein(), x, y)

### 收纳

In [139]:
# 读取数据
merged_triples = NTuple{3, String}[]
signs = NTuple{3, String}[]
open("ILLs_fusion/merged_ILLs_KG_en_zh.txt", "r") do io
    while true
        line = readline(io)
        isempty(line) && break
        triple, sign = get_triple_sign(line)
        push!(merged_triples, triple)
        push!(signs, sign)
    end
end
zh_triples_ILLs = merged_triples[162545:end]
zh_signs = signs[162545:end]

# 写入数据
# extract_io = open("extract/merged_en_zh_byILLs.txt", "w")
# open("ILLs_fusion/merged_ILLs_KG_en_zh.txt", "r") do io
#     while (line = readline(io)) != ""
#         triple, sign = get_triple_sign(line)
#         println(extract_io, join(triple, '\t'))
#     end
# end

In [160]:
# 中文存在一对多的情况
downloadILLs = Tuple{String, String}[]
for (raw, new, sign) in zip(zh_triples, zh_triples_ILLs, zh_signs)
    new[1] == new[3] && continue
    sign[1] == "en" && new[1] != raw[1] && push!(downloadILLs, (new[1], raw[1]))
    sign[3] == "en" && new[3] != raw[3] && push!(downloadILLs, (new[3], raw[3]))
end
unique!(downloadILLs) # 58556

54197-element Vector{Tuple{String, String}}:
 ("Sichuan", "四川省")
 ("Chengdu", "成都市")
 ("Faye_Wong", "王菲")
 ("Yuan_Shikai", "袁世凯")
 ("Yikuang,_Prince_Qing", "奕劻")
 ("Lou_Tseng-Tsiang", "陸徵祥")
 ("Li_Yuanhong", "黎元洪")
 ("René_Descartes", "勒内·笛卡尔")
 ("French_people", "法兰西人")
 ("Plato", "柏拉图")
 ("Aristotle", "亚里士多德")
 ("Augustine_(disambiguation)", "奥古斯丁")
 ("Arnos", "阿尔诺")
 ⋮
 ("2004_UEFA_Intertoto_Cup", "2004年歐洲足協圖圖盃")
 ("Naypyidaw_Safari_Park", "内比都狩猎旅行公园")
 ("MacBook_(Retina)", "MacBook_(Retina显示屏)")
 ("Enoch_Derant_Lakoué", "埃諾克·德朗·拉庫埃")
 ("Qaraqosh_Protection_Committee", "克拉克斯保护委员会")
 ("2009_UEFA_European_Under-19_Championship", "2009年歐洲U-19足球錦標賽")
 ("Denmark_women's_national_handball_team", "丹麥國家女子手球隊")
 ("François_Bozizé", "弗朗索瓦·博齐泽")
 ("S.V._Zulte_Waregem", "聚尔特瓦勒海姆体育俱乐部")
 ("K.V._Kortrijk", "皇家哥積克足球會")
 ("Kenny_Smith", "肯尼·史密斯")
 ("Art_Gallery_of_Ontario", "安大略美术馆")

In [171]:
# 合并后的实体对齐
open("ILLs/combine_ills.txt", "w") do io
    for (en, zh) in sort(unique!(vcat(downloadILLs, ILLs)), by=last)
        println(io, en, '\t', zh)
    end
end

In [174]:
# 不在 ILLs 中的实体对齐
open("ILLs/new_ills.txt", "w") do io
    for (en, zh) in sort(filter(∉(Set(ILLs)), downloadILLs), by=last)
        println(io, en, '\t', zh)
    end
end

In [35]:
dlILLs = Dict{String, String}()
en_subs = Set(first.(en_triples))
for ele in zip(zh_triples, zh_triples_ILLs, signs), (raw, new, sign) in zip(ele...)
    sign == "en" && isascii(new) && (dlILLs[raw] = new)
end

In [39]:
fails = Tuple{String, String}[]
for (key, val) in ILLs_zh_en
    (haskey(dlILLs, key) && dlILLs[key] == val) || push!(fails, (key, val))
end

In [60]:
en_objs = Set(last.(en_triples))
en_subs = Set(first.(en_triples))

Set{String} with 86608 elements:
  "1914–15_NHA_season"
  "TOMfest"
  "Xenophon_Overton_Pindall"
  "Jingzhou"
  "Vyškov_District"
  "Bajmok"
  "Peter_Brown_(rugby_league)"
  "Tiszabercel"
  "Catherine_Ardagh"
  "La_Bazoque,_Calvados"
  "Glades_County,_Florida"
  "Bihor_County"
  "Araksavan"
  "Ruth_Worsley"
  "Le_Mans"
  "Luxembourg_Stock_Exchange"
  "Beatriz_Corredor"
  "Montgomery,_Alabama"
  "Witowy_Most"
  "Győrzámoly"
  "Game_6_of_the_1998_NBA_Finals"
  "Piątkowa,_Lesser_Poland_Voivodeship"
  "USNS_Mission_Capistrano_(T-AO-112)"
  "Benedictine_University"
  "Halifax,_Nova_Scotia"
  ⋮ 

In [67]:
necc_ILLs, necc_triples = Set{Tuple{String, String}}(), NTuple{3, String}[]
for (raw, new, sign) in zip(zh_triples, zh_triples_ILLs, signs)
    # 英文实体，头接尾，尾接头
    flag = false
    if sign[1] == "en" && !isascii(raw[1]) && new[1] ∈ en_objs
        tmp = (raw[1], new[1])
        tmp ∈ necc_ILLs || (push!(necc_ILLs, tmp); flag=true)
    end
    if sign[3] == "en" && !isascii(raw[3]) && new[3] ∈ en_subs
        tmp = (raw[3], new[3])
        tmp ∈ necc_ILLs || (push!(necc_ILLs, tmp); flag=true)
    end
    flag && push!(necc_triples, raw)
end

In [74]:
words = unique!(vcat(first.(zh_triples), last.(zh_triples_ILLs)))
println("中文实体总数:\t", length(words))
non_ILLs = filter(∉(keys(ILLs_zh_en)), words)
println("非 ILLs 对齐实体数目:\t", length(non_ILLs))


中文实体总数:	87031
非 ILLs 对齐实体数目:	75790


In [68]:
necc_ILLs
"约翰内斯堡大学" => "University of Johannesburg"
"華特迪士尼工作室電影" => "Walt Disney Studios Electric"

Set{Tuple{String, String}} with 20940 elements:
  ("戴立克的進化", "Evolution_of_the_Daleks")
  ("2014年南美自由盃", "Estadio_Defensores_del_Chaco")
  ("紹斯波特", "Southport")
  ("阿尔塔薛西斯二世", "Artaxerxes_II_of_Persia")
  ("宋嘉澍", "Charlie_Soong")
  ("恩納省", "Enna")
  ("玛丽亚·费奥多罗芙娜皇后_(保罗一世)", "Maria_Feodorovna_(Sophie_Dorothea_of_…
  ("尼古拉斯_(希腊王子)", "Prince_Nikolaos_of_Greece_and_Denmark")
  ("约翰内斯堡大学", "University_of_Johannesburg")
  ("華特迪士尼工作室電影", "Walt_Disney_Studios_Motion_Pictures")
  ("禤智輝", "Geoff_Hoon")
  ("師利·奧羅賓多", "Sri_Aurobindo")
  ("南美洲足球協會", "CONMEBOL")
  ("里克·斯科特", "Charlie_Crist")
  ("奥洛特", "西班牙行政区划")
  ("八打雁省", "Batangas")
  ("奥克兰_(加利福尼亚州)", "Oakland,_California")
  ("佛罗里达州", "Florida")
  ("西烏克蘭人民共和國", "Ukraine")
  ("格拉涅纳德莱斯加里格斯", "Granyena_de_les_Garrigues")
  ("路易斯·博塔", "Pretoria")
  ("苏尔坦·本·阿卜杜勒-阿齐兹·阿勒沙特", "Sultan_bin_Abdulaziz")
  ("皮乌拉", "Piura")
  ("小野寺五典", "Itsunori_Onodera")
  ("金永南", "Kim_Yong-nam")
  ⋮ 

In [42]:
for (key, val) in fails
    haskey(dlILLs, key) && println(val, '\t', dlILLs[key])
end

Deçan	Districts_of_Kosovo
Farsta_(borough)	Postal_codes_in_Sweden
Ciechanowiec	Gmina
Wejherowo	Powiat
Hill_County,_Montana	Havre,_Montana
Ken's_Bar_II	Jazz
Thomas_Gold	Ithaca,_New_York
FK_Voždovac	Belgrade
Apies_River	South_Africa
Scott_Moir	London,_Ontario
Mahamat_Kamoun	Central_African_Republic
Albert_Lutuli	African_National_Congress
Kep_Province	Districts_of_Cambodia
Éamon_de_Valera	Cathal_Brugha
Flag_of_Lancashire	Friends_of_Real_Lancashire
Adult_Swim	Cartoon_Network
Peugeot_208	Volkswagen_Polo_R_WRC
Sheridan_County,_North_Dakota	McClusky,_North_Dakota
El_Escorial	Juan_Bautista_de_Toledo
2014_FIFA_World_Cup_qualification_–_CONCACAF_Second_Round	Guatemala_City
2011_Rugby_World_Cup	Westpac_Stadium
Fayette_County,_Georgia	Fayetteville,_Georgia
Young_Money_Entertainment	Mack_Maine
Atlantic_Star	Arctic_Star
E=MC²_(Mariah_Carey_album)	Taiwan
Kurenai_(song)	Marc_Bolan
Rise_of_the_Guardians	Alec_Baldwin
Illumos	Unix
Leduc,_Alberta	Environment_Canada
Villeneuve-sur-Lot	Aquitaine
Hrant_Bagra

In [48]:
filter(i->!haskey(ILLs_zh_en, i[1]), dlILLs)

Dict{String, String} with 42868 entries:
  "美格福斯"          => "Magnavox"
  "德索托縣_(佛羅里達… => "DeSoto_County,_Florida"
  "1934年世界盃外圍賽… => "1934_FIFA_World_Cup_qualification"
  "教宗斯德望一世"    => "Pope_Stephen_I"
  "阿尔比区"          => "Arrondissement_of_Albi"
  "弗萊明_(紐約州)"   => "Fleming,_New_York"
  "巴利语"            => "Pali"
  "戴夫·格鲁辛"       => "Dave_Grusin"
  "康斯坦丁诺斯·德米… => "Konstantinos_Demertzis"
  "利根川"            => "Tone_River"
  "蒙克斯科纳_(南卡罗… => "Moncks_Corner,_South_Carolina"
  "泰赖什凯"          => "Tereske"
  "狐狸與獵狗2"       => "The_Fox_and_the_Hound_2"
  "Bernie_Brillstein" => "Bernie_Brillstein"
  "普勒斯顿_(肯塔基州… => "Prestonsburg,_Kentucky"
  "塔斯馬尼亞大學"    => "University_of_Tasmania"
  "绍塞迪利亚"        => "Saucedilla"
  "确认死亡"          => "Confirmed_Dead"
  "馬鞍山_(香港市鎮)" => "Ma_On_Shan_(town)"
  "萨克森王国"        => "Kingdom_of_Saxony"
  "贝尔纳多特王朝"    => "House_of_Bernadotte"
  "天主教奥三棉示总教… => "Roman_Catholic_Archdiocese_of_Ozamiz"
  "菲鷹航空541號班機… => "Air_Philippines_Flight_541"
  "易卜拉欣·纳西尔"   => "

In [40]:
# write_xlsx("excel_data/ILLs.xlsx", permutedims(hcat(collect.(ILLs)...)))

In [38]:
modify(txt) = lowercase(filter(i->isletter(i) || isdigit(i), txt))
ILLs_MT = permutedims(read_xlsx("excel_data/ILLs_MT.xlsx"))
ILLs_MT = [(modify(a), modify(b)) for (a, b) in eachcol(ILLs_MT)]
fails = filter(i->i[1]!=i[2], ILLs_MT)

8319-element Vector{Tuple{String, String}}:
 ("guixers", "jixels")
 ("westvoorne", "westforne")
 ("ronjohnsonbaseball", "ronjohnsonbaseballplayer")
 ("maychan", "chenjiajia")
 ("catherineofnavarre", "katherinenavarra")
 ("pyinoolwindistrict", "pyinoolwincounty")
 ("cyranoagency", "bignoseloverloveoperationgroup")
 ("qianlongemperor", "emperorqianlong")
 ("princephilippedukeoforléans18691926", "philipdukeoforleans")
 ("thomasckinkaid", "thomaskinkade")
 ("197677europeancup", "197677europeanchampionscup")
 ("thegoodthebadandtheugly", "twilightthree")
 ("parkhillskentucky", "parkhillkentucky")
 ⋮
 ("ptolemyxalexanderi", "ptolemyx")
 ("28thhongkongfilmawards", "the28thhongkongfilmawards")
 ("khamtaisiphandon", "candysipanton")
 ("socialwelfareconstituency", "socialwelfarefunctionalconstituency")
 ("navoiy", "navoi")
 ("inbeepark", "parkinbee")
 ("listofpremiersofbermuda", "primeministerofbermuda")
 ("bìnhđịnhprovince", "binhdinhprovince")
 ("evacheng", "zhengruhua")
 ("stevenpoon", "panguo